

![banner](https://drive.google.com/uc?export=view&id=1QT4HBhpAMakPzvhj2NLrZQ29EOPuLEB7)



---


# Description

This notebook will read in files in the folder /Data in this directory to create the event map for this website. Please note that this does not check for the file extension. Files should be .csv.

Please download a .csv file to inspect its format.

  * Lines 0-1: Used for customizing the symbol color <br>
  * Line    2: header for data<br>
  * Lines 3-n: data  <br>

# Folium Maps

Default map tiles is: openstreetmap

Other free types:
* mapquestopen
* MapQuest Open Aerial
* Mapbox Bright
* Mapbox Control Room
* stamenterrain (USA only)
* stamentoner
* stamenwatercolor
* cartodbpositron
* cartodbdark_matter

Proprietary map types may also be used.

# Known issues

1. 
Currently the legend is not generated automatically. Additional entires must be made to the legend and a new map generated. Auto generation could be done with javascript as well as converting this notebook to leaflet.js.



# Authors

Lorraine J. Hwang, UC Davis <br>
Denise Kwong, UC Davis

---

In [1]:
# Import the folium library
import folium

# Use pandas
import pandas as pd

# load library to access operating system functionalities
import os

In [2]:
# read in all data from the data directory
  
# assign path
path, dirs, files = next(os.walk("./Data/"))
file_count = len(files)

# create empty list
dataframes_list = []
files_basename = []
symbol_color = []

# next websafe color is: #99cc33

for i in range(file_count):
# read in the second line to get the symbol color. Note this does not error check.
    temp_color = pd.read_csv("./Data/"+files[i], nrows=1)
    symbol_color.append(temp_color)
# append datasets to the list skipping over the first 2 lines
    temp_df = pd.read_csv("./Data/"+files[i], dtype={'HHMMSS':str, 'HHMM':str}, skiprows=2)
#    temp_df = pd.read_csv("./Data/"+files[i], skiprows=2)
    dataframes_list.append(temp_df)
# get filename without extension
    basename_without_extension = os.path.splitext(os.path.basename(files[i]))[0]
    files_basename.append(basename_without_extension)

# debugging
    print(files[i])
    print(files_basename[i])
    print(symbol_color[i])
    print(dataframes_list[i])     
# display datasets
#for dataset in dataframes_list:
#    display(dataset)


    


SCEC.csv
SCEC
  symbol_color
0      #f36f20
      YYYYMMDD  HHMMSS   Latitude   Longitude  Magnitude
0     19630109   60403  34.952167 -119.177000        4.1
1     19630112  195233  32.000000 -116.401333        3.8
2     19630113   23939  33.017000 -116.216833        4.3
3     19630113  204518  33.025667 -116.178833        3.9
4     19630114   13330  33.022500 -116.189333        3.6
...        ...     ...        ...         ...        ...
2082  19911204   71057  33.069667 -116.803500        4.3
2083  19911204   81703  34.177667 -117.022333        4.1
2084  19911220   31930  35.531667 -117.374667        3.5
2085  19911220  103829  35.535000 -117.371333        4.2
2086  19911226  164107  35.550500 -117.410333        3.6

[2087 rows x 5 columns]
Seismo_Archive.csv
Seismo_Archive
  symbol_color
0      #8e499b
    YYYYMMDD  HHMM  Latitude  Longitude  Depth  Magnitude
0   19060418  1312    38.000    -123.00    NaN       7.90
1   19060817    10    51.000     179.00    NaN       7.80
2   19060

In [5]:
# Import from folium library
from folium import FeatureGroup
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# Set the default map location
map = folium.Map(location=[20,50],  zoom_start=2, world_copy_jump=True, no_wrap=False)

# Add map layer options
folium.raster_layers.TileLayer("Cartodb Positron").add_to(map)
#folium.raster_layers.TileLayer("Stamen Watercolor").add_to(map)
folium.raster_layers.TileLayer("Stamen Terrain").add_to(map)


# Loop over each dataframe
for j in range(file_count):
    data_feature = FeatureGroup(name=files_basename[j])
    data_cluster = MarkerCluster().add_to(data_feature)
    
    #  Add logic for the different timestamps. Note leading zereos are not preserved in popup.
    if 'HHMMSS' in dataframes_list[j]:
        str_timing = 'HHMMSS'
    elif 'HHMM' in dataframes_list[j]:
        str_timing = 'HHMM'
    else:
        print('timing does not conform to HHMM or HHMMSS format')
 #   print (j, str_timing) 
    
    # Create an inner loop Iterate through the events and create circle markers for each
    for i in range(0,len(dataframes_list[j])):
        mag = dataframes_list[j].iloc[i]['Magnitude']
        # color_outline, color_fill = change_color(mag)
        # Create the popup for each marker according to the data
        # First add in the leading 0's to the time
        formatted_time = dataframes_list[j].iloc[i][str_timing]
        if (str_timing == 'HHMM' and len(dataframes_list[j].iloc[i][str_timing]) < 4):
            while(len(formatted_time) != 4):
                formatted_time = "0" + formatted_time
        elif (str_timing == 'HHMMSS' and len(dataframes_list[j].iloc[i][str_timing]) < 6):
            while(len(formatted_time) != 6):
                formatted_time = "0" + formatted_time

        html=f"""
            <p style="font-family:Arial;font-size:75%;">
                <strong>Magnitude:</strong> {dataframes_list[j].iloc[i]['Magnitude']}<br>
               <strong>Date (YYYYMMDD):</strong> {int(dataframes_list[j].iloc[i]['YYYYMMDD'])} <br>
                <strong>Time ({str_timing}):</strong> {formatted_time} <br>
                <strong>Latitude:</strong> {dataframes_list[j].iloc[i]['Latitude']}<br>
                <strong>Longitude:</strong> {dataframes_list[j].iloc[i]['Longitude']}
            </p>
        
            """
        iframe = folium.IFrame(html=html, width=200, height=80)
        popup = folium.Popup(iframe, max_width=2650)
        # Create CircleMarkers for each event with radius equal to the magnitude of the event
        # Location taken from data, tooltip equal to the magnitude of the event
       
        temp_color=symbol_color[j].iloc[0]['symbol_color']
      
        # print(temp_color)
        folium.CircleMarker(
                location=[dataframes_list[j].iloc[i]['Latitude'], dataframes_list[j].iloc[i]['Longitude']],
                popup=popup,
                tooltip = mag,
                # color=color_outline,
                # fill_color=color_fill,
                color=temp_color,
                fill_color=temp_color,
                opacity = 1,
                fill_opacity = 1,
                radius= mag
        ).add_to(data_cluster)
        
        # Add feature groups 
        # data_cluster.add_to(map)  
    data_feature.add_to(map)
        
# Add layer control to map
folium.LayerControl().add_to(map)

# Using HTML to create legend based on color of Markers
# Create rectangle shapes and include labels for each
# This is ugly but I have no good solution currently other than using javascript
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
    <body>

        <div id='maplegend' class='maplegend' 
            style='position: absolute; z-index:500; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
             border-radius:6px; padding: 10px; font-size:12px; right: 20px; bottom: 20px;'>
     
        <div class='legend-title' style='text-align:center;'><b>Legend</b></div>
            <div><div class ='SCSN'></div> SCSN</div>
            <div><div class ='Seismo_Archive'></div> Seismo Archives</div>
            <div><div class ='WWSSN'></div> WWSSN M>6</div>
        </div>

    </body>
</html>

<style type='text/css'>
    .SCSN {
      height: 15px;
      width: 30px;
      background-color: #f36f20;
      display:inline-block;
    }
    .Seismo_Archive {
      height: 15px;
      width: 30px;
      background-color: #8e499b;
      display:inline-block;
    }
    .WWSSN {
      height: 15px;
      width: 30px;
      background-color: #007ac2;
      display:inline-block;
    }
</style>

{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

map.get_root().add_child(macro)

# Display within the notebook
#map
# Write to file
map.save('events.html')